In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [2]:
    classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
    # config file 들고오기
    cfg = Config.fromfile('/opt/ml/code/mmdetection_trash/configs/_base_/models/swim_base_htc.py')

    PREFIX = '../../input/data/'
    # dataset 바꾸기
    cfg.data.train.classes = classes
    cfg.data.train.img_prefix = PREFIX
    cfg.data.train.ann_file = PREFIX + 'train_data1.json'
    cfg.data.train.seg_prefix=PREFIX  
    #cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

    cfg.data.val.classes = classes
    cfg.data.val.img_prefix = PREFIX
    cfg.data.val.ann_file = PREFIX + 'valid_data1.json'
    #cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

    cfg.data.test.classes = classes
    cfg.data.test.img_prefix = PREFIX
    cfg.data.test.ann_file = PREFIX + 'test.json'
    #cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

    cfg.data.samples_per_gpu = 2

    # cfg.optimizer = dict(type='AdamW', lr=0.0001, weight_decay=0.0001)
    # cfg.lr_config = dict(
    #     policy='CosineRestart',
    #     warmup='linear',
    #     warmup_iters=200,
    #     warmup_ratio=1.0 / 10,
    #     periods=[20,12,12,12,12],
    #     restart_weights = [1,0.7,0.6,0.5,0.4],
    #     min_lr_ratio=2e-6)

    cfg.seed= 24
    cfg.gpu_ids = [0]
    cfg.work_dir = './work_dirs/swim_b_htc_01_final'

    cfg.checkpoint_config = dict(max_keep_ckpts=2, interval=1)
    #cfg.model.roi_head.bbox_head.num_classes = 11

    #이어하기
    #cfg.resume_from = './work_dirs/swin_t_mask_rcnn/epoch_15.pth'
    cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

    #group_name = '아무거나'; ##그룹이름(run 여러개를 그룹으로 묶어줄 수 있습니다)
    project_name = 'stage_3_object_det'; ##프로젝트 이름
    run_name = 'swim_b_htc_01_final'; ##run 이름stage_3_object_det
    config_list = {
        'epoch' : cfg.runner.max_epochs,
        'batch_size' :  cfg.data.samples_per_gpu,
        'optimizer' : cfg.optimizer,
        'optimizer_config' : cfg.optimizer_config,
        'lr_config' : cfg.lr_config
    }
    ## 원하는 wandb.init() argument값 추가해주세요
    #cfg.log_config.hooks[1].init_kwargs['group']=group_name   # group name(option)
    cfg.log_config.hooks[1].init_kwargs['project'] = project_name
    cfg.log_config.hooks[1].init_kwargs['name'] = run_name    # run name
    cfg.log_config.hooks[1].init_kwargs['config'] = config_list    # config

In [3]:
model = build_detector(cfg.model)

2021-05-19 14:29:34,786 - mmdet - INFO - load model from: /opt/ml/code/mmdetection_trash/pretrained/swin_base_patch4_window7_224_22k.pth
2021-05-19 14:29:36,006 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.42s)
creating index...
index created!
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2611: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-19 14:29:44,709 - mmdet - INFO - Start running, host: root@a1eb508df260, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/swim_b_htc_01_final
2021-05-19 14:29:44,710 - mmdet - INFO - workflow: [('train', 1)], max: 60 epochs
/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=1.18s)
creating index...
index created!
wandb: Currently logged in as: openingsound (use `wandb login --relogin` to force relogin)


2021-05-19 14:30:06,158 - mmdet - INFO - Epoch [1][10/1306]	lr: 6.099e-07, eta: 1 day, 7:55:12, time: 1.467, data_time: 0.288, memory: 15385, loss_rpn_cls: 0.6907, loss_rpn_bbox: 0.1077, loss_semantic_seg: 14.7969, s0.loss_cls: 3.1680, s0.acc: 7.5098, s0.loss_bbox: 0.0126, s0.loss_mask: 5.9262, s1.loss_cls: 0.6056, s1.acc: 89.3359, s1.loss_bbox: 0.0032, s1.loss_mask: 4.2694, s2.loss_cls: 1.2436, s2.acc: 7.3145, s2.loss_bbox: 0.0001, s2.loss_mask: 3.0878, loss: 33.9117, grad_norm: 563.8508
2021-05-19 14:30:19,015 - mmdet - INFO - Epoch [1][20/1306]	lr: 1.277e-06, eta: 1 day, 5:56:55, time: 1.286, data_time: 0.017, memory: 15385, loss_rpn_cls: 0.6898, loss_rpn_bbox: 0.0625, loss_semantic_seg: 13.0237, s0.loss_cls: 2.4181, s0.acc: 10.8398, s0.loss_bbox: 0.0075, s0.loss_mask: 5.2629, s1.loss_cls: 0.4316, s1.acc: 97.9297, s1.loss_bbox: 0.0024, s1.loss_mask: 3.8108, s2.loss_cls: 1.0218, s2.acc: 8.3203, s2.loss_bbox: 0.0001, s2.loss_mask: 2.6478, loss: 29.3792, grad_norm: 502.1407
2021-05-19 

KeyboardInterrupt: 

In [11]:
filename = '../../input/data/batch_03/0749.png'
filename = filename[:-9]+'_masks'+filename[-9:]
filename

'../../input/data/batch_03_masks/0749.png'